In [5]:
import nipype.pipeline.engine as pe
from nipype.interfaces import ants
from nipype.interfaces import fsl
from nipype.interfaces import io as nio

In [11]:
templates = {'struct':'/home/gdholla1/data/daphne/clean/{subject_id}/t1.nii.gz'}

workflow = pe.Workflow(name='register2mni_daphne', base_dir='/home/gdholla1/workflow_folders/')

selector = pe.Node(nio.SelectFiles(templates), name='selector')
selector.iterables = [('subject_id', ['S00'])]

better = pe.Node(fsl.BET(), name='better')
workflow.connect(selector, 'struct', better, 'in_file')

reg = pe.Node(ants.Registration(), name='antsRegister')
reg.inputs.transforms = ['Rigid', 'Affine', 'SyN']
reg.inputs.transform_parameters = [(0.1,), (0.1,), (0.1, 3.0, 0.0)]
reg.inputs.number_of_iterations = [[1000,500,250,100]]*2 + [[100,100,70,20]]
reg.inputs.dimension = 3
reg.inputs.write_composite_transform = True
reg.inputs.collapse_output_transforms = True
reg.inputs.metric = ['MI']*2 + ['CC']
reg.inputs.metric_weight = [1]*3 # Default (value ignored currently by ANTs)
reg.inputs.radius_or_number_of_bins = [32]*2 + [4]
reg.inputs.sampling_strategy = ['Regular']*2 + [None]
reg.inputs.sampling_percentage = [0.25]*2 + [None]
reg.inputs.convergence_threshold = [1.e-8]*2 + [1e-9]
reg.inputs.convergence_window_size = [10]*2 + [15]
reg.inputs.smoothing_sigmas = [[3,2,1,0]]*3
reg.inputs.sigma_units = ['mm']*3
reg.inputs.shrink_factors = [[8,4,2,1]]*2 + [[6,4,2,1]]
reg.inputs.use_estimate_learning_rate_once = [True, True, True]
reg.inputs.use_histogram_matching = [False]*2 + [True] # This is the default
reg.inputs.initial_moving_transform_com = True
reg.inputs.output_warped_image = True
reg.inputs.winsorize_lower_quantile = 0.01
reg.inputs.winsorize_upper_quantile = 0.99

reg.inputs.fixed_image = '/usr/share/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz'

workflow.connect(better, 'out_file', reg, 'moving_image')

ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = '/home/gdholla1/data/daphne/registration/'

workflow.connect(better, 'out_file', ds, 't1_weighted_bet')
workflow.connect(reg, 'composite_transform', ds, 't1_weighted2mni.@mat')
workflow.connect(reg, 'warped_image', ds, 't1_weighted2mni')
workflow.connect(reg, 'inverse_composite_transform', ds, 'mni2t1_weighted')

workflow.run()

In [9]:
reg.outputs


composite_transform = <undefined>
forward_invert_flags = <undefined>
forward_transforms = <undefined>
inverse_composite_transform = <undefined>
inverse_warped_image = <undefined>
reverse_invert_flags = <undefined>
reverse_transforms = <undefined>
save_state = <undefined>
warped_image = <undefined>